In [7]:
import pandas as pd

print("--- DIAGNOSTIC DE CORRESPONDANCE DES NOMS  ---")

# 1. Charger  fichier CLIMAT (GADM)
# Assure-toi que ce fichier est bien créé par ton étape précédente
file_climate = "FLDAS_Nigeria_Admin2_Aggregated.csv"

try:
    df_clim = pd.read_csv(file_climate)
    print(f"Fichier Climat chargé ({len(df_clim)} lignes)")
    
    # 2. Charger  fichier IPC (CIBLE)
    
    file_ipc = "IPC.xlsx" 
    
    
   
    try:
        df_ipc = pd.read_excel(file_ipc) 
        print(f"Fichier IPC chargé ({len(df_ipc)} lignes)")

        # --- PARAMÈTRES ---
        col_ipc_name = 'adm2_name'   # LE VRAI NOM (Trouvé grâce à ton fichier)
        col_clim_name = 'NAME_2'     # Standard GADM
        
        # 3. La Comparaison
        # On met tout en MAJUSCULES et on enlève les espaces vides autour (.strip())
        set_clim = set(df_clim[col_clim_name].astype(str).str.upper().str.strip())
        set_ipc = set(df_ipc[col_ipc_name].astype(str).str.upper().str.strip())

        match = set_clim.intersection(set_ipc)
        only_clim = set_clim - set_ipc # Ce qu'on a en météo mais pas en IPC
        only_ipc = set_ipc - set_clim  # Ce qu'on a en IPC mais pas en météo

        print(f"\n--- RÉSULTAT DU MATCHING ---")
        print(f"Régions uniques Climat (GADM) : {len(set_clim)}")
        print(f"Régions uniques IPC (WFP)     : {len(set_ipc)}")
        print("-" * 30)
        print(f"MATCH PARFAIT : {len(match)} régions")
        print(f"PROBLÈMES     : {len(only_clim) + len(only_ipc)} régions ne correspondent pas")
        
        # Calcul du score
        score = len(match) / len(set_ipc) * 100
        print(f" Taux de couverture IPC : {score:.1f}%")

        if len(only_ipc) > 0:
            print("\nExemples de régions IPC introuvables côté Climat (à renommer) :")
            print(list(only_ipc)[:10])

    except Exception as e:
        print(f"Erreur lecture IPC : {e}")

except Exception as e:
    print(f"Erreur lecture Climat : {e}")

--- DIAGNOSTIC DE CORRESPONDANCE DES NOMS  ---
Fichier Climat chargé (43620 lignes)
Fichier IPC chargé (12185 lignes)

--- RÉSULTAT DU MATCHING ---
Régions uniques Climat (GADM) : 724
Régions uniques IPC (WFP)     : 568
------------------------------
MATCH PARFAIT : 429 régions
PROBLÈMES     : 434 régions ne correspondent pas
 Taux de couverture IPC : 75.5%

Exemples de régions IPC introuvables côté Climat (à renommer) :
['BIRNIN-GWARI', 'OGU/BOLO', 'REMO NORTH', 'YAGBA WEST', 'ABA NORTH', 'MUBI NORTH', 'OWAN WEST', 'KOSOFE', 'BARIKIN LADI', 'ENUGU EAST']


In [9]:
import pandas as pd
import difflib

# 1. Chargement des fichiers
# Adapter les noms de fichiers si nécessaire
file_climate = "FLDAS_Nigeria_Admin2_Aggregated.csv"
file_ipc = "IPC.xlsx" 

try:
    df_clim = pd.read_csv(file_climate)
    df_ipc = pd.read_csv(file_ipc) 

    # 2. Extraction des listes de régions
    # Nettoyage des noms (majuscules + suppression espaces) pour comparer
    gadm_names = sorted(list(set(df_clim['NAME_2'].astype(str).str.upper().str.strip())))
    ipc_names = sorted(list(set(df_ipc['adm2_name'].astype(str).str.upper().str.strip())))

    # 3. Identification des orphelins (Ceux qui sont dans IPC mais pas dans GADM)
    only_ipc = [name for name in ipc_names if name not in gadm_names]

    print(f"Recherche de correspondances pour {len(only_ipc)} régions...")

    # 4. Algorithme de correspondance (Fuzzy Matching)
    print("\n--- Dictionnaire de correction suggéré ---")
    print("correction_dict = {")
    
    for name in only_ipc:
        # On cherche le nom le plus proche dans la liste GADM
        # cutoff=0.6 signifie qu'il faut au moins 60% de ressemblance
        matches = difflib.get_close_matches(name, gadm_names, n=1, cutoff=0.6)
        
        if matches:
            print(f'    "{name}": "{matches[0]}",')
        else:
            # Si aucune ressemblance trouvée
            print(f'    # "{name}": "???", # Aucune correspondance trouvée')
            
    print("}")

except Exception as e:
    print(f"Erreur : {e}")

Erreur : 'utf-8' codec can't decode bytes in position 15-16: invalid continuation byte


In [16]:
import pandas as pd
import difflib

# Configuration
file_climate = "FLDAS_Nigeria_Admin2_Aggregated.csv"
file_ipc = "IPC.xlsx" # Assure-toi que c'est le bon nom (avec .xlsx)

print(f"--- Chargement du fichier Excel : {file_ipc} ---")

# 1. Chargement du fichier Climat
try:
    df_clim = pd.read_csv(file_climate)
    print(f"✅ Climat chargé : {len(df_clim)} lignes.")
except Exception as e:
    print(f"❌ Erreur Climat : {e}")
    df_clim = None

# 2. Chargement du fichier IPC (VERSION EXCEL)
try:
    # ⚠️ C'est ici que ça change : on utilise read_excel !
    df_ipc = pd.read_excel(file_ipc)
    print(f"✅ IPC chargé : {len(df_ipc)} lignes.")
    
    # Nettoyage des noms de colonnes
    df_ipc.columns = df_ipc.columns.str.strip()
    
except Exception as e:
    print(f"❌ Erreur IPC : {e}")
    print("Vérifie que le fichier s'appelle bien 'IPC.xlsx' et qu'il n'est pas ouvert dans Excel.")
    df_ipc = None

# 3. Création du Dictionnaire
if df_clim is not None and df_ipc is not None:
    # On cherche la colonne Admin2
    col_ipc = 'adm2_name'
    if col_ipc not in df_ipc.columns:
        print(f"⚠️ Colonne '{col_ipc}' introuvable. Colonnes dispo : {list(df_ipc.columns)}")
        # Essaye de trouver une colonne qui ressemble
        candidats = [c for c in df_ipc.columns if "adm2" in c.lower() or "area" in c.lower()]
        if candidats:
            col_ipc = candidats[0]
            print(f"👉 Utilisation de la colonne : {col_ipc}")

    if col_ipc in df_ipc.columns:
        gadm_names = sorted(list(set(df_clim['NAME_2'].astype(str).str.upper().str.strip())))
        ipc_names = sorted(list(set(df_ipc[col_ipc].astype(str).str.upper().str.strip())))

        only_ipc = [n for n in ipc_names if n not in gadm_names]

        print(f"\n--- DICTIONNAIRE DE CORRECTION ({len(only_ipc)} régions à mapper) ---")
        print("correction_dict = {")
        
        for name in only_ipc:
            matches = difflib.get_close_matches(name, gadm_names, n=1, cutoff=0.6)
            if matches:
                print(f'    "{name}": "{matches[0]}",')
            else:
                pass 
                
        print("}")
        print("-" * 30)

--- Chargement du fichier Excel : IPC.xlsx ---
✅ Climat chargé : 43620 lignes.
✅ IPC chargé : 12185 lignes.

--- DICTIONNAIRE DE CORRECTION (139 régions à mapper) ---
correction_dict = {
    "ABA NORTH": "AWKANORTH",
    "ABA SOUTH": "ABASOUTH",
    "ABEOKUTA NORTH": "ABEOKUTANORTH",
    "ABEOKUTA SOUTH": "ABEOKUTASOUTH",
    "ADO EKITI": "ADO-EKITI",
    "ADO-ODO/OTA": "ADOODO/OTA",
    "AGEGE": "NANGERE",
    "AHOADA EAST": "AHOADAEAST",
    "AHOADA WEST": "AHOADAWEST",
    "AJEROMI-IFELODUN": "IFELODUN",
    "AKUKU TORU": "AKUKU-TORU",
    "AMUWO-ODOFIN": "AMUWOODOFIN",
    "AREWA-DANDI": "AREWADANDI",
    "BAKASSI": "TAKAI",
    "BARIKIN LADI": "BARKINLADI",
    "BEKWARA": "BEKWARRA",
    "BENUE": "BENDE",
    "BIDA": "BINDAWA",
    "BIRNI KUDU": "BIRNINKUDU",
    "BIRNIN KEBBI": "BIRNINKEBBI",
    "BIRNIN MAGAJI": "BIRNINGWARI",
    "BIRNIN-GWARI": "BIRNINGWARI",
    "BURSARI": "BORSARI",
    "BURUKU": "BURUTU",
    "CALABAR SOUTH": "CALABARSOUTH",
    "CHANCHAGA": "SHANGA",
    "

In [18]:
import pandas as pd

# 1. Chargement
file_climate = "FLDAS_Nigeria_Admin2_Aggregated.csv"
file_ipc = "IPC.xlsx"

df_clim = pd.read_csv(file_climate)
df_ipc = pd.read_excel(file_ipc)

# =================================================================
# 👇 C'EST ICI QUE ÇA MANQUAIT ! ON MET LE CLIMAT EN MAJUSCULES 👇
df_clim['NAME_2'] = df_clim['NAME_2'].astype(str).str.upper().str.strip()
# =================================================================

# 2. Nettoyage IPC (Comme avant)
df_ipc.columns = df_ipc.columns.str.strip()

# Ton dictionnaire nettoyé (celui où tu as enlevé les bêtises)
correction_dict = {
    "ABA NORTH": "AWKANORTH",
    "ABA SOUTH": "ABASOUTH",
    "ABEOKUTA NORTH": "ABEOKUTANORTH",
    "ABEOKUTA SOUTH": "ABEOKUTASOUTH",
    "ADO EKITI": "ADO-EKITI",
    "ADO-ODO/OTA": "ADOODO/OTA",
    "AHOADA EAST": "AHOADAEAST",
    "AHOADA WEST": "AHOADAWEST",
    "AJEROMI-IFELODUN": "IFELODUN",
    "AKUKU TORU": "AKUKU-TORU",
    "AMUWO-ODOFIN": "AMUWOODOFIN",
    "AREWA-DANDI": "AREWADANDI",
    "BARIKIN LADI": "BARKINLADI",
    "BIRNIN KEBBI": "BIRNINKEBBI",
    "BIRNIN-GWARI": "BIRNINGWARI",
    "CALABAR SOUTH": "CALABARSOUTH",
    "DAWAKIN KUDU": "DAWAKINKUDU",
    "DAWAKIN TOFA": "DAWAKINTOFA",
    "DUTSIN-MA": "DUTSINMA",
    "EKITI EAST": "EKITIEAST",
    "EKITI SOUTH WEST": "EKITISOUTH-WEST",
    "EKITI WEST": "EKITIWEST",
    "ENUGU EAST": "ENUGUEAST",
    "ENUGU NORTH": "ENUGUNORTH",
    "ESAN CENTRAL": "ESANCENTRAL",
    "ESAN NORTH EAST": "ESANNORTH-EAST",
    "ESAN SOUTH EAST": "ESANSOUTH-EAST",
    "ESAN WEST": "ESANWEST",
    "ETSAKO CENTRAL": "ETSAKOCENTRAL",
    "ETSAKO EAST": "ETSAKOEAST",
    "ETSAKO WEST": "ETSAKOWEST",
    "GWER EAST": "GWEREAST",
    "GWER WEST": "GWERWEST",
    "IBEJU/LEKKI": "IBEJU-LEKKI",
    "IGBO-EZE NORTH": "IGBO-EZENORTH",
    "IGBO-EZE SOUTH": "IGBO-EZESOUTH",
    "IJEBU EAST": "IJEBUEAST",
    "IJEBU NORTH": "IJEBUNORTH",
    "IJEBU NORTH EAST": "IJEBUNORTH-EAST",
    "IJEBU ODE": "IJEBU-ODE",
    "ILORIN EAST": "ILORINEAST",
    "ILORIN SOUTH": "ILORINSOUTH",
    "ILORIN WEST": "ILORINWEST",
    "ISIALANGWA NORTH": "ISIALANGWANORTH",
    "ISIALANGWA SOUTH": "ISIALANGWASOUTH",
    "JOS EAST": "JOSEAST",
    "JOS NORTH": "JOSNORTH",
    "JOS SOUTH": "JOSSOUTH",
    "KADUNA NORTH": "KADUNANORTH",
    "KARIM-LAMIDO": "KARIMLAMIDO",
    "KAURA NAMODA": "KAURANAMODA",
    "LAGOS ISLAND": "LAGOSISLAND",
    "LANGTANG NORTH": "LANGTANGNORTH",
    "LANGTANG SOUTH": "LANGTANGSOUTH",
    "MUBI NORTH": "MUBINORTH",
    "MUBI SOUTH": "MUBISOUTH",
    "NASARAWA-EGGON": "NASARAWAEGGON",
    "NKANU EAST": "NKANUEAST",
    "NKANU WEST": "NKANUWEST",
    "OBIA/AKPOR": "OBIO/AKPOR",
    "OTURKPO": "OTUKPO",
    "OVIA NORTH EAST": "OVIANORTH-EAST",
    "OVIA SOUTH WEST": "OVIASOUTH-WEST",
    "OWAN EAST": "OWANEAST",
    "OWAN WEST": "OWANWEST",
    "PORT-HARCOURT": "PORTHARCOURT",
    "QUA'AN PAN": "QUA'ANPAN",
    "REMO NORTH": "REMO-NORTH",
    "RIMIN GADO": "RIMINGADO",
    "SABON BIRNI": "SABONBIRNI",
    "SABON-GARI": "SABONGARI",
    "SOKOTO NORTH": "SOKOTONORTH",
    "SULE-TANKARKAR": "SULETANKARKAR",
    "TALATA MAFARA": "TALATA-MAFARA",
    "TUDUN WADA": "TUDUNWADA",
    "UKWA EAST": "UKWAEAST",
    "UKWA WEST": "UKWAWEST",
    "UMUAHIA NORTH": "UMUAHIANORTH",
    "UMUAHIA SOUTH": "UMUAHIASOUTH",
    "UMUNNEOCHE": "UMUNNEOCHI",
    "YAGBA EAST": "YAGBAEAST",
    "YAGBA WEST": "YAGBAWEST",
    "YEWA NORTH": "YEWANORTH",
    "YEWA SOUTH": "YEWASOUTH",
    "YOLA NORTH": "YOLANORTH",
    "YOLA SOUTH": "YOLASOUTH",
    "ZANGO-KATAF": "ZANGONKATAF"
}

# Application sur IPC
df_ipc['adm2_clean'] = df_ipc['adm2_name'].replace(correction_dict)
df_ipc['adm2_clean'] = df_ipc['adm2_clean'].astype(str).str.upper().str.strip()

# 3. Fusion
print("Fusion en cours...")
df_final_dataset = pd.merge(
    df_ipc, 
    df_clim, 
    left_on='adm2_clean', 
    right_on='NAME_2', 
    how='inner'
)

# 4. Sauvegarde
output_file = "FINAL_DATASET_NIGERIA.csv"
df_final_dataset.to_csv(output_file, index=False)

print(f"SUCCÈS ! Dataset final créé : {len(df_final_dataset)} lignes !")

Fusion en cours...
SUCCÈS ! Dataset final créé : 579960 lignes !


In [19]:
import pandas as pd

# Chargement du fichier final
df_final = pd.read_csv("FINAL_DATASET_NIGERIA.csv")

print("--- PREUVE DE L'HARMONISATION ---")

# 1. Vérification du nombre de zones
nb_regions = df_final['NAME_2'].nunique()
print(f"Nombre de districts (LGA) uniques dans le fichier final : {nb_regions}")
print("(Le Nigéria compte environ 774 LGA. Si tu es entre 700 et 774, c'est excellent).")

# 2. Inspection visuelle de la 'soudure'
# On regarde le nom original (IPC) vs le nom officiel (Climat) sur la même ligne
cols_a_verifier = ['adm2_name', 'adm2_clean', 'NAME_2', 'phase35', 'SoilMoisture00_10cm_tavg']

# On prend 5 lignes au hasard où il y a des données
sample = df_final[cols_a_verifier].sample(5)

print("\n--- Échantillon de 5 lignes jointes ---")
print(sample)

C:\Users\valen\AppData\Local\Temp\ipykernel_22216\2800732199.py:4: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_final = pd.read_csv("FINAL_DATASET_NIGERIA.csv")


--- PREUVE DE L'HARMONISATION ---
Nombre de districts (LGA) uniques dans le fichier final : 445
(Le Nigéria compte environ 774 LGA. Si tu es entre 700 et 774, c'est excellent).


KeyError: "['SoilMoisture00_10cm_tavg'] not in index"

In [20]:
import pandas as pd

# 1. On affiche tous les noms de colonnes disponibles
print("--- LISTE OFFICIELLE DES COLONNES ---")
cols = df_final.columns.tolist()
for c in cols:
    print(f"👉 {c}")

print("\n" + "-"*30)

# 2. On affiche les 5 premières lignes sans filtrer (pour voir les données)
print("--- APERÇU DU FICHIER FINAL ---")
print(df_final.head())

--- LISTE OFFICIELLE DES COLONNES ---
👉 adm0_name
👉 adm0_pcod2
👉 adm0_pcod3
👉 adm0_5_name
👉 adm0_5_pcod2
👉 adm1_name
👉 adm1_pcod2
👉 adm1_5_name
👉 adm1_5_pcod2
👉 adm2_name
👉 adm2_pcod2
👉 adm3_pcod2
👉 exercise_code
👉 exercise_label
👉 exercise_year
👉 chtype
👉 population
👉 phase_class
👉 phase1
👉 phase2
👉 phase3
👉 phase4
👉 phase5
👉 phase35
👉 adm2_clean
👉 time
👉 NAME_1
👉 NAME_2
👉 Evap_tavg
👉 LWdown_f_tavg
👉 Lwnet_tavg
👉 Psurf_f_tavg
👉 Qair_f_tavg
👉 Qg_tavg
👉 Qh_tavg
👉 Qle_tavg
👉 Qs_tavg
👉 Qsb_tavg
👉 RadT_tavg
👉 Rainf_f_tavg
👉 SWE_inst
👉 SWdown_f_tavg
👉 SnowCover_inst
👉 SnowDepth_inst
👉 Snowf_tavg
👉 Swnet_tavg
👉 Tair_f_tavg
👉 Wind_f_tavg
👉 SoilMoi00_10cm_tavg
👉 SoilMoi10_40cm_tavg
👉 SoilMoi40_100cm_tavg
👉 SoilMoi100_200cm_tavg
👉 SoilTemp00_10cm_tavg
👉 SoilTemp10_40cm_tavg
👉 SoilTemp40_100cm_tavg
👉 SoilTemp100_200cm_tavg

------------------------------
--- APERÇU DU FICHIER FINAL ---
  adm0_name adm0_pcod2 adm0_pcod3  adm0_5_name  adm0_5_pcod2 adm1_name  \
0   Nigeria         NG        NGA    

In [ ]:
import pandas as pd

# 1. Chargement du fichier fusionné
input_file = "FINAL_DATASET_NIGERIA.csv"
df = pd.read_csv(input_file)

print(f"Chargement de {len(df)} lignes et {len(df.columns)} colonnes.")

# 2. Suppression des doublons techniques uniquement
# 'adm2_clean' : servait juste de pont pour la fusion
# 'NAME_1', 'NAME_2' : ce sont les noms GADM, redondants avec adm1_name/adm2_name
cols_techniques_a_supprimer = ['adm2_clean', 'NAME_1', 'NAME_2']

# On supprime seulement si elles existent
df_full = df.drop(columns=[c for c in cols_techniques_a_supprimer if c in df.columns])

# 3. Renommage optionnel pour plus de clarté (sans supprimer de données)
# On rend juste les noms techniques de la NASA plus lisibles
mapping_climat = {
    'time': 'Date',
    'Rainf_f_tavg': 'Rainfall',
    'Tair_f_tavg': 'Air_Temp',
    'Wind_f_tavg': 'Wind_Speed',
    'Qair_f_tavg': 'Specific_Humidity',
    'Psurf_f_tavg': 'Surface_Pressure',
    'SoilMoi00_10cm_tavg': 'Soil_Moisture_00_10cm',
    'SoilMoi10_40cm_tavg': 'Soil_Moisture_10_40cm',
    'SoilMoi40_100cm_tavg': 'Soil_Moisture_40_100cm',
    'SoilMoi100_200cm_tavg': 'Soil_Moisture_100_200cm',
    'SoilTemp00_10cm_tavg': 'Soil_Temp_00_10cm'
}

df_full = df_full.rename(columns=mapping_climat)

# 4. Sauvegarde
output_filename = "NIGERIA_FINAL_FULL.csv"
df_full.to_csv(output_filename, index=False)

print("-" * 30)
print(f"Fichier COMPLET sauvegardé : {output_filename}")
print(f"Il contient {len(df_full.columns)} variables (Climat + IPC).")
print("Prêt pour le push GitHub.")

C:\Users\valen\AppData\Local\Temp\ipykernel_22216\3164256472.py:5: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(input_file)


Chargement de 579960 lignes et 56 colonnes.
------------------------------
✅ Fichier COMPLET sauvegardé : NIGERIA_FINAL_FULL.csv
Il contient 53 variables (Climat + IPC).
Prêt pour le push GitHub.


In [ ]:
#transformation en parquet 
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

csv_file = "FINAL_DATASET_NIGERIA.csv"
parquet_file = "FINAL_DATASET_NIGERIA.parquet"
chunksize = 100_000

# échantillon pour construire un schéma stable
sample = pd.read_csv(csv_file, nrows=5000)

# Convertir explicitement les colonnes "suspectes" en string
for c in sample.columns:
    if "name" in c.lower() or "pcod" in c.lower():
        sample[c] = sample[c].astype("string")

schema = pa.Table.from_pandas(sample, preserve_index=False).schema

writer = pq.ParquetWriter(parquet_file, schema)

for chunk in pd.read_csv(csv_file, chunksize=chunksize):
    for c in chunk.columns:
        if "name" in c.lower() or "pcod" in c.lower():
            chunk[c] = chunk[c].astype("string")

    table = pa.Table.from_pandas(chunk, preserve_index=False).cast(schema)
    writer.write_table(table)

writer.close()


C:\Users\valen\AppData\Local\Temp\ipykernel_13604\1748375043.py:21: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(csv_file, chunksize=chunksize):
C:\Users\valen\AppData\Local\Temp\ipykernel_13604\1748375043.py:21: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(csv_file, chunksize=chunksize):


In [3]:
df = pd.read_parquet("FINAL_DATASET_NIGERIA.parquet")
df.head()


,adm0_name,adm0_pcod2,adm0_pcod3,adm0_5_name,adm0_5_pcod2,adm1_name,adm1_pcod2,adm1_5_name,adm1_5_pcod2,adm2_name,...,Tair_f_tavg,Wind_f_tavg,SoilMoi00_10cm_tavg,SoilMoi10_40cm_tavg,SoilMoi40_100cm_tavg,SoilMoi100_200cm_tavg,SoilTemp00_10cm_tavg,SoilTemp10_40cm_tavg,SoilTemp40_100cm_tavg,SoilTemp100_200cm_tavg
0,Nigeria,NG,NGA,<NA>,<NA>,Gombe,NG16,Gombe Central,NG01601,Akko,...,295.44380,7.376234,0.153800,0.346961,0.352089,0.368408,296.12460,296.21597,296.43713,296.90494
1,Nigeria,NG,NGA,<NA>,<NA>,Gombe,NG16,Gombe Central,NG01601,Akko,...,298.26970,8.111750,0.147464,0.336445,0.341699,0.358832,299.19684,298.30210,297.60660,297.21780
2,Nigeria,NG,NGA,<NA>,<NA>,Gombe,NG16,Gombe Central,NG01601,Akko,...,302.62866,5.717865,0.149106,0.330567,0.335564,0.351844,303.43622,302.03384,300.51343,298.98935
3,Nigeria,NG,NGA,<NA>,<NA>,Gombe,NG16,Gombe Central,NG01601,Akko,...,301.19107,5.073179,0.203690,0.329308,0.333594,0.346686,302.72183,302.36606,301.78183,300.82278
4,Nigeria,NG,NGA,<NA>,<NA>,Gombe,NG16,Gombe Central,NG01601,Akko,...,299.87973,4.916486,0.292827,0.326689,0.329967,0.342687,300.94012,301.21283,301.40800,301.26984


In [4]:
#fonction qui convertit les csv en parquet

from __future__ import annotations

from pathlib import Path
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq


def csv_to_parquet(
    csv_path: str | Path,
    parquet_path: str | Path | None = None,
    chunksize: int = 100_000,
    *,
    sep: str = ",",
    encoding: str | None = None,
    low_memory: bool = False,
    text_cols_contains: tuple[str, ...] = ("name", "pcod"),
) -> Path:
    """
    Convertit un CSV en Parquet en lecture par chunks (adapté aux gros fichiers).
    - Stabilise le schéma Parquet sur le 1er chunk.
    - Force en 'string' les colonnes dont le nom contient certains mots-clés (par défaut: name, pcod),
      pour éviter les changements de dtype entre chunks.
    """
    csv_path = Path(csv_path)
    if parquet_path is None:
        parquet_path = csv_path.with_suffix(".parquet")
    parquet_path = Path(parquet_path)

    writer: pq.ParquetWriter | None = None
    schema: pa.Schema | None = None

    read_kwargs = dict(
        sep=sep,
        chunksize=chunksize,
        low_memory=low_memory,
    )
    if encoding is not None:
        read_kwargs["encoding"] = encoding

    try:
        for chunk in pd.read_csv(csv_path, **read_kwargs):
            # Forcer les colonnes texte "instables" en string (ex: *_name, *_pcod*)
            if text_cols_contains:
                for c in chunk.columns:
                    cl = c.lower()
                    if any(k in cl for k in text_cols_contains):
                        chunk[c] = chunk[c].astype("string")

            table = pa.Table.from_pandas(chunk, preserve_index=False)

            if writer is None:
                schema = table.schema
                writer = pq.ParquetWriter(parquet_path, schema)

            # Cast vers le schéma initial pour garantir la stabilité
            table = table.cast(schema)
            writer.write_table(table)

    finally:
        if writer is not None:
            writer.close()

    return parquet_path



In [9]:
from pathlib import Path

folder = Path(r"C:\Users\valen\OneDrive\Documents\GitHub\Projet-de-Statapp---WFP-\Dataset creation - NASA Nigeria")  # mets ton dossier ici
for csv_file in folder.glob("*.csv"):
    out = csv_to_parquet(csv_file)
    print(csv_file.name, "->", out.name)


FINAL_DATASET_NIGERIA.csv -> FINAL_DATASET_NIGERIA.parquet
FLDAS_Nigeria_Admin2_Aggregated.csv -> FLDAS_Nigeria_Admin2_Aggregated.parquet
FLDAS_Nigeria_pixels.csv -> FLDAS_Nigeria_pixels.parquet
FLDAS_Nigeria_pixels_full.csv -> FLDAS_Nigeria_pixels_full.parquet
geodata_FLDAS_Nigeria.csv -> geodata_FLDAS_Nigeria.parquet
geodata_FLDAS_Nigeria_full.csv -> geodata_FLDAS_Nigeria_full.parquet
NIGERIA_FINAL_FULL.csv -> NIGERIA_FINAL_FULL.parquet
